In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Generate synthetic data
input_size = 1000
output_size = 10
num_samples = 100000

# Create random input and output data
X = torch.randn(num_samples, input_size).to(device)
y = torch.randint(0, output_size, (num_samples,)).to(device)

# Create a simple neural network
class StressTestNet(nn.Module):
    def __init__(self):
        super(StressTestNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, output_size)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Instantiate the model
model = StressTestNet().to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create DataLoader for batching
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

# Train the model
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(dataloader):
        # Zero the gradient buffers
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (batch_idx + 1) % 10 == 0:
            print(f"Epoch [{epoch + 1}/{epochs}], Step [{batch_idx + 1}/{len(dataloader)}], Loss: {running_loss / 10:.4f}")
            running_loss = 0.0

print("Stress test complete!")
